<a href="https://colab.research.google.com/github/SadeemAlasiri/week9/blob/main/Copy_of_RAG_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

## Step 1: Install Required Libraries

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [3]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 5.80MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [4]:
!pip install chromadb sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 6.6 MB/s eta 0:00

In [5]:
!pip install langchain

  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


# Step 2: Load the Traffic Violations Dataset

In [1]:
import pandas as pd

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [6]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [8]:
print(df.columns)

Index(['المخالفة', 'الغرامة'], dtype='object')


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [9]:
markdown_list = []

for index, row in df.iterrows():
    violation = row['المخالفة']
    fine = row['الغرامة']

    markdown_text = f"{violation} - {fine} ريال"
    markdown_list.append(markdown_text)

for text in markdown_list[:5]:
    print(text)

قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة المالية 100 - 150 ريال ريال
ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. - الغرامة المالية 100 - 150 ريال ريال
عدم وجود تأمين ساري للمركبة. - الغرامة المالية 100 - 150 ريال ريال
عبور المشاة للطرق من غير الأماكن المخصصة لهم. - الغرامة المالية 100 - 150 ريال ريال
عدم تقيد المشاة بالإشارات الخاصة بهم. - الغرامة المالية 100 - 150 ريال ريال


## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [30]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10
)

In [31]:
text=text_splitter.create_documents(markdown_list)

In [32]:
text

[Document(metadata={}, page_content='قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة المالية 100 - 150 ريال ريال'),
 Document(metadata={}, page_content='ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. - الغرامة المالية 100 - 150 ريال ريال'),
 Document(metadata={}, page_content='عدم وجود تأمين ساري للمركبة. - الغرامة المالية 100 - 150 ريال ريال'),
 Document(metadata={}, page_content='عبور المشاة للطرق من غير الأماكن المخصصة لهم. - الغرامة المالية 100 - 150 ريال ريال'),
 Document(metadata={}, page_content='عدم تقيد المشاة بالإشارات الخاصة بهم. - الغرامة المالية 100 - 150 ريال ريال'),
 Document(metadata={}, page_content='وقوف المركبة في أماكن غير مخصصة للوقوف. - الغرامة المالية 100 - 150 ريال ريال'),
 Document(metadata={}, page_content='عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم. - الغرامة المالية 100 - 150'),
 Document(metadata={}, page_content='100 - 150 ريال ريال'),
 Document(metadata={}, page_content='عدم استخدام إشارة الالتفاف عند التحول لليمي

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [14]:
!pip install -U langchain-community

  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00


In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


https://huggingface.co/aubmindlab/bert-base-arabertv02

In [16]:
embedding_model = HuggingFaceEmbeddings(model_name="aubmindlab/bert-base-arabertv02")

<ipython-input-16-b494b6b198a2>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(model_name="aubmindlab/bert-base-arabertv02")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
vector_store = Chroma(collection_name="traffic", embedding_function=embedding_model)


<ipython-input-17-b624d28dfd6a>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(collection_name="traffic", embedding_function=embedding_model)


In [18]:
texts = [doc.page_content for doc in text]
vector_store.add_texts(texts=texts)
print(f" إضافة {len(texts)} الغرمات")

 إضافة 104 الغرمات


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [19]:
from langchain.prompts import PromptTemplate

In [20]:
template = """
 تقديم معلومات عن المخالفات المرورية في السعودية. المخالفات

المخالفه: {context}

السؤال: {query}

"""

In [21]:
rag_prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=template
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [25]:
!pip install langchain-groq

  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.3 MB/s eta 0:00:00


In [26]:
from langchain_groq import ChatGroq

In [27]:
from google.colab import userdata
groq_api_key=userdata.get('Groq')

In [28]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192",
    temperature=0
)

In [29]:
query = "ما هي الغرامة ترك المركبة في طريق منحدرة مع عدم اتخاذ الاحتياطات اللازمة في السعوديه ؟"

response = llm.invoke([{'role':'user', 'content': query}])
print(response)

content='في المملكة العربية السعودية، الغرامة التي تُقابل ترك المركبة في طريق منحدرة مع عدم اتخاذ الاحتياطات اللازمة هي 500 ريال سعودي (حوالي 133 دولار أمريكي) according to the Saudi Traffic Law.\n\nالمادة 134 من قانون المرور السعوديstates that:\n\n"من يترك مركبته في طريق منحدرة أو طريق سريع أو طريق معبر دون اتخاذ الاحتياطات اللازمة لسلامة المركبة أو السائحين، يُقابل بالغرامة المقررة في المادة 134 من هذا القانون، والتي تبلغ 500 ريال."\n\nTranslation:\n"Whoever leaves their vehicle on a sloping road or a high-speed road or a crossing road without taking the necessary precautions for the safety of the vehicle or the passengers, shall be fined according to Article 134 of this law, which is 500 Riyals."\n\nIt\'s worth noting that the fine may be increased if the driver\'s negligence causes an accident or harm to others. Additionally, the driver may also face other penalties, such as suspension of their driving license or imprisonment, depending on the severity of the incident.' additional_

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [33]:
!pip install langchain

In [34]:
from langchain.chains import LLMChain

In [35]:
MODEL = LLMChain(llm=llm,
                 prompt=rag_prompt,
                 verbose=True)

<ipython-input-35-8bd533300a72>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  MODEL = LLMChain(llm=llm,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [36]:
def query_rag(user_query):

    retrieved_context = vector_store.similarity_search(user_query, k=1)
    context = retrieved_context[0].page_content

    generated_answer = MODEL.run({
        "context": context,
        "query": user_query
    })

    return generated_answer

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [37]:
user_query = "ما هي الغرامة على القيادة بدون رخصة "
answer = query_rag(user_query)
print("الإجابة:", answer)

<ipython-input-36-ec6f9096a9c4>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  generated_answer = MODEL.run({


Prompt after formatting:

 تقديم معلومات عن المخالفات المرورية في السعودية. المخالفات

المخالفه: قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة المالية 100 - 150 ريال ريال

السؤال: ما هي الغرامة على القيادة بدون رخصة 



> Finished chain.
الإجابة: In Saudi Arabia, driving without a license is considered a serious traffic violation and is punishable by law. Here are the details:

**Traffic Violation:** Driving a vehicle in areas where driving is not permitted.

**Fine:** SR 100-150 (approximately USD 27-40)

**Driving without a License:**

* Fine: SR 500-1,000 (approximately USD 133-267)
* Suspension of license for a period of 1-3 months
* Possible imprisonment for a period of 1-3 months

It's important to note that driving without a license is a serious offense in Saudi Arabia and can result in severe penalties, including imprisonment. It's essential to obtain a valid driver's license and follow all traffic laws and regulations to avoid such penalties.

Additionally, it